# Library

In [ ]:
!pip install peft ipywidgets bitsandbytes

In [241]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import load_dataset
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchinfo import summary
from torch.utils.data import Dataset
import huggingface_hub
from transformers import (
    AutoTokenizer,
    AutoModel,
    AutoModelForCausalLM,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    BitsAndBytesConfig,
)
from peft import (
    get_peft_model,
    AutoPeftModel,
    AutoPeftModelForCausalLM,
    PeftModel,
    LoraConfig,
    LoftQConfig,
    TaskType,
)

# Lightweight

기존 학습된 모델의 정확도를 유지하면서 모델 크기를 줄이고, 연산을 간소화하는 기법.

<br>

<font style="font-size:18px"> 종류 </font>

- Pruning
- Quantization
- Knowledge Distillation
- PEFT

## PEFT

![](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2Fb76aDK%2FbtsHuMdZpAK%2FWCBvKZOfiEwhWsJlpOhKQ1%2Fimg.jpg)

reference: https://huggingface.co/docs/peft/v0.6.0/index

<br>

PEFT (Parameterized Efficient Fine-Tuning): 모델을 보다 효율적으로 fine-tuning하기 위한 기술. <br>

### Adapter Tuning

기존 모델의 특정 레이어에 작은 네트워크(adapter)를 삽입하여 fine-tuning. <br>
모델의 기본 parameter는 고정하고 adapter만 새로운 task에 적합하도록 함. <br>

<br>

> ```python
> class AdapterLayer(nn.Module):
>     def __init__(self, input_dim, output_dim, reduction_factor=2):
>         super().__init__()
> 
>         self.downsample = nn.Linear(input_dim, input_dim//reduction_factor)
>         self.activation = nn.ReLU()
>         self.upsample = nn.Linear(input_dim//reduction_factor, output_dim)
> 
>     def forward(self, x):
>         x = self.downsample(x)
>         x = self.activation(x)
>         x = self.upsample(x)
> 
>         return x
> 
> class BertWithAdapter(nn.Module):
>     def __init__(self, model_name, reduction_factor):
>         super().__init__()
> 
>         self.bert = BertModel.from_pretrained(model_name)
>         self.adapters = nn.ModuleList([
>             AdapterLayer(self.bert.config.hidden_size, self.bert.config.hidden_size, reduction_factor)
>             for _ in range(self.bert.config.num_hidden_layers)
>         ])
> 
>     def forward(self, inputs):
>         outputs = self.bert(**inputs)
>         hidden_states = outputs.last_hidden_state
>         
>         for i, adapter in enumerate(self.adapters):
>             hidden_states = adapter(hidden_states)
>         
>         return hidden_states
> ```

In [32]:
model_name = 'bert-base-uncased'
# 뉴스에 대한 감성분석 모델
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [36]:
class DiscussionReviewLayer(nn.Module):  # nn.Module을 상속하여 커스텀 레이어 클래스 생성
    def __init__(self, in_feature, out_feature, num_labels):
        super().__init__()  # 부모 클래스의 초기화 함수 호출

        # 첫 번째 선형 변환: 입력 차원(in_feature)에서 중간 출력 차원(out_feature)으로 매핑
        self.linear = nn.Linear(in_feature, out_feature)
        
        # 비선형 활성화 함수 ReLU 정의
        self.relu = nn.ReLU()
        
        # 최종 출력 선형 변환: 중간 출력(out_feature)에서 레이블 수(num_labels)로 매핑
        self.out = nn.Linear(out_feature, num_labels)
    
    def forward(self, x):
        # 입력 x에 대해 첫 번째 선형 변환 수행
        x = self.linear(x)
        
        # ReLU 활성화 함수 적용
        x = self.relu(x)
        
        # 두 번째 선형 변환으로 최종 예측 출력 생성
        x = self.out(x)

        return x  # 최종 출력 반환

In [49]:
class PaperReviewLayer(nn.Module):  # nn.Module을 상속하여 PaperReviewLayer 클래스 생성
    def __init__(self, in_feature, out_feature, num_labels):
        super().__init__()  # nn.Module의 초기화 함수 호출

        # 첫 번째 선형 변환: 입력 차원(in_feature)에서 중간 출력 차원(out_feature)으로 매핑
        self.linear = nn.Linear(in_feature, out_feature)
        
        # 비선형 활성화 함수 ReLU 정의
        self.relu = nn.ReLU()
        
        # 두 번째 선형 변환: 중간 출력 차원(out_feature)에서 레이블 수(num_labels)로 매핑
        self.out = nn.Linear(out_feature, num_labels)
    
    def forward(self, x):
        # 입력 텐서 x를 첫 번째 선형 변환에 통과
        x = self.linear(x)
        
        # 활성화 함수 ReLU 적용
        x = self.relu(x)
        
        # 두 번째 선형 변환 적용
        x = self.out(x)

        return x  # 최종 출력 반환

In [50]:
# PaperReviewLayer 클래스를 사용하여 모델의 분류기(classifier) 레이어를 새로운 레이어로 교체하는 코드입니다.

# 입력 특징 수 768, 중간 출력 차원 768, 레이블 수 3으로 PaperReviewLayer 클래스의 인스턴스를 생성
paper_review_layer = PaperReviewLayer(768, 768, 3)

# 생성한 PaperReviewLayer 인스턴스를 모델의 classifier 레이어에 할당하여 교체
# 이를 통해 모델의 분류기 부분이 새로 정의된 PaperReviewLayer 구조로 대체됩니다.
model.classifier = paper_review_layer

In [54]:
torch.save(model.classifier.state_dict(), 'paper_review_layer.pth')

In [61]:
paper_review_layer.load_state_dict(torch.load('paper_review_layer.pth'))

C:\Users\USER\AppData\Local\Temp\ipykernel_23548\3416307944.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  paper_review_layer.load_state_dict(torch.load('paper_review_l

<All keys matched successfully>

In [71]:
class PaperReviewBert(nn.Module):
    def __init__(
            self,
            in_feature,
            out_feature,
            num_labels,
            model_name='bert-base-uncased',
        ):
        super().__init__()
        # 사전 학습된 BERT 모델을 불러옴 (분류를 위한 BERT 모델)
        self.bert = AutoModelForSequenceClassification.from_pretrained(model_name).bert
        # PaperReviewLayer를 초기화하여 최종 출력 레이어 설정
        self.paper_review_layer = PaperReviewLayer(in_feature, out_feature, num_labels)

    def forward(self, x):
        # BERT 모델을 통해 입력을 전파하고, [CLS] 토큰의 출력만 가져옴
        x = self.bert(x).last_hidden_state[:, 0, :]
        # PaperReviewLayer를 통해 출력 변환
        x = self.paper_review_layer(x)

        return x

In [366]:
# practice (Colab, L4)
# 데이터 출처
## 네이버 영화: https://github.com/e9t/nsmc/
## 네이버 쇼핑: https://github.com/bab2min/corpus/tree/master/sentiment

# 모델: bert-base-multilingual-uncased, llama
# 네이버 영화와 네이버 쇼핑에 대한 감성 분류 모델을 각각 학습합니다.

# 영화 데이터 로드
movie_data = pd.read_csv(
    './data/naver_movie.txt',
    sep='\t',
    usecols=['document', 'label']  # 문서와 레이블 열만 사용
)

# 쇼핑 데이터 로드
shopping_data = pd.read_csv(
    './data/naver_shopping.txt',
    sep='\t',
    header=None,
    names=['label', 'review']  # 열 이름 지정
)

# 쇼핑 데이터 레이블 값 확인
shopping_data.label.value_counts()

# 레이블 값 변경: 1, 2, 4, 5를 0, 1, 2, 3으로 변환
shopping_data.label = shopping_data.label.replace({1: 0, 2: 1, 4: 2, 5: 3})

# NaverMoviewClassifier 클래스 정의
class NaverMoviewClassifer(nn.Module):
    def __init__(self, in_feature, out_feature, num_labels):
        super().__init__()
        # 선형 레이어, ReLU 활성화 함수 및 최종 출력 레이어 정의
        self.linear = nn.Linear(in_feature, out_feature)
        self.relu = nn.ReLU()
        self.out = nn.Linear(out_feature, num_labels)

    def forward(self, x):
        # 입력을 선형 레이어에 통과시키고 활성화
        x = self.linear(x)
        x = self.relu(x)
        # 최종 출력
        x = self.out(x)
        return x

# NaverMovieBert 클래스 정의
class NaverMovieBert(nn.Module):
    def __init__(
            self,
            in_feature,
            out_feature,
            num_labels,
            model_name='bert-base-uncased',
        ):
        super().__init__()
        # BERT 모델 로드 (분류를 위한 BERT)
        self.bert = AutoModelForSequenceClassification.from_pretrained(model_name).bert
        # 영화 분류기 레이어 정의
        self.naver_movie_classifier = NaverMoviewClassifer(in_feature, out_feature, num_labels)

        # BERT의 모든 파라미터를 고정(freeze)
        for parameter in self.bert.parameters():
            parameter.requires_grad = False

    def forward(self, input_ids=None, token_type_ids=None, attention_mask=None, labels=None):
        # BERT 모델에 입력 전달
        outputs = self.bert(
            input_ids=input_ids,
            token_type_ids=token_type_ids,
            attention_mask=attention_mask
        )
        
        # BERT의 [CLS] 토큰 출력으로 분류 수행
        logits = self.naver_movie_classifier(outputs.last_hidden_state[:, 0, :])

        # 손실을 계산하고 반환
        loss = None
        if labels is not None:
            loss = F.cross_entropy(logits, labels)

        return (loss, logits) if loss is not None else logits

# NaverShoppingClassifier 클래스 정의
class NaverShoppingClassifier(nn.Module):
    def __init__(self, in_feature, out_feature, num_labels):
        super().__init__()
        # 선형 레이어, ReLU 활성화 함수 및 최종 출력 레이어 정의
        self.linear = nn.Linear(in_feature, out_feature)
        self.relu = nn.ReLU()
        self.out = nn.Linear(out_feature, num_labels)

    def forward(self, x):
        # 입력을 선형 레이어에 통과시키고 활성화
        x = self.linear(x)
        x = self.relu(x)
        # 최종 출력
        x = self.out(x)
        return x

# NaverShoppingBert 클래스 정의
class NaverShoppingBert(nn.Module):
    def __init__(
            self,
            in_feature,
            out_feature,
            num_labels,
            model_name='bert-base-uncased',
        ):
        super().__init__()
        # BERT 모델 로드
        self.bert = AutoModel.from_pretrained(model_name)
        # 쇼핑 분류기 레이어 정의
        self.naver_shopping_classifier = NaverShoppingClassifier(in_feature, out_feature, num_labels)

        # BERT의 모든 파라미터를 고정(freeze)
        for parameter in self.bert.parameters():
            parameter.requires_grad = False

    def forward(self, input_ids=None, token_type_ids=None, attention_mask=None, labels=None):
        # BERT 모델에 입력 전달
        outputs = self.bert(
            input_ids=input_ids,
            token_type_ids=token_type_ids,
            attention_mask=attention_mask
        )
        
        # BERT의 [CLS] 토큰 출력으로 분류 수행
        logits = self.naver_shopping_classifier(outputs.last_hidden_state[:, 0, :])

        # 손실을 계산하고 반환
        loss = None
        if labels is not None:
            loss = F.cross_entropy(logits, labels)

        return (loss, logits) if loss is not None else logits

# 데이터 샘플 수를 2000으로 제한
movie_data = movie_data.iloc[:2000]
shopping_data = shopping_data.iloc[:2000]

In [367]:
# 사용할 모델 이름 정의 (Google BERT의 다국어 버전)
model_name = 'google-bert/bert-base-multilingual-uncased'

# AutoTokenizer를 사용하여 지정한 모델에 맞는 토크나이저를 로드
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [368]:
# 영화 리뷰 데이터에 대해 토큰화 및 필요한 처리를 적용
movie_data['tokenized'] = movie_data.document.apply(lambda x: tokenizer(
    x,                                # 입력 텍스트
    padding='max_length',            # 최대 길이에 맞춰 패딩 추가
    max_length=128,                  # 최대 토큰 수 (128로 설정)
    truncation=True,                 # 길이가 초과하는 경우 잘림 처리
    return_tensors='pt',            # PyTorch 텐서 형식으로 반환
))

# 쇼핑 리뷰 데이터에 대해 토큰화 및 필요한 처리를 적용
shopping_data['tokenized'] = shopping_data.review.apply(lambda x: tokenizer(
    x,                                # 입력 텍스트
    padding='max_length',            # 최대 길이에 맞춰 패딩 추가
    max_length=128,                  # 최대 토큰 수 (128로 설정)
    truncation=True,                 # 길이가 초과하는 경우 잘림 처리
    return_tensors='pt',            # PyTorch 텐서 형식으로 반환
))

In [369]:
import pandas as pd
from torch.utils.data import Dataset

class NaverDataset(Dataset):
    def __init__(self, data):
        """
        데이터셋 초기화

        Args:
            data (pd.DataFrame): 입력 데이터프레임
        """
        self.data = data  # 데이터프레임을 인스턴스 변수로 저장
    
    def __len__(self):
        """
        데이터셋의 길이를 반환

        Returns:
            int: 데이터셋의 샘플 수
        """
        return len(self.data)  # 데이터프레임의 길이 반환

    def __getitem__(self, idx):
        """
        주어진 인덱스에 해당하는 샘플을 반환

        Args:
            idx (int): 샘플의 인덱스

        Returns:
            dict: 입력 데이터와 레이블을 포함하는 딕셔너리
        """
        temp = self.data.iloc[idx]  # 주어진 인덱스의 데이터를 가져옴

        return {
            'input_ids': temp.tokenized.get('input_ids')[0],         # 토큰화된 입력 ID
            'token_type_ids': temp.tokenized.get('token_type_ids')[0], # 토큰 유형 ID
            'attention_mask': temp.tokenized.get('attention_mask')[0], # 어텐션 마스크
            'labels': temp.label,                                       # 레이블
        }

In [370]:
# 쇼핑 데이터셋을 훈련, 검증, 테스트 세트로 분할
train_shopping, temp = train_test_split(shopping_data, test_size=0.5, random_state=0)
valid_shopping, test_shopping = train_test_split(temp, test_size=0.5, random_state=0)

# 영화 데이터셋을 훈련, 검증, 테스트 세트로 분할
train_movie, temp = train_test_split(movie_data, test_size=0.5, random_state=0)
valid_movie, test_movie = train_test_split(temp, test_size=0.5, random_state=0)

# NaverDataset 클래스를 사용하여 데이터셋 객체 생성
train_dataset_shopping = NaverDataset(train_shopping)  # 훈련 쇼핑 데이터셋
valid_dataset_shopping = NaverDataset(valid_shopping)  # 검증 쇼핑 데이터셋
test_dataset_shopping = NaverDataset(test_shopping)    # 테스트 쇼핑 데이터셋

train_dataset_movie = NaverDataset(train_movie)      # 훈련 영화 데이터셋
valid_dataset_movie = NaverDataset(valid_movie)      # 검증 영화 데이터셋
test_dataset_movie = NaverDataset(test_movie)        # 테스트 영화 데이터셋

In [ ]:
# 각각의 데이터로 adapter 학습 후 weight 저장

# 훈련 매개변수를 설정합니다.
training_args = TrainingArguments(
    output_dir='./results',                   # 결과 저장 디렉토리
    eval_strategy='epoch',                    # 평가 전략: 에포크마다 평가
    learning_rate=2e-5,                       # 학습률
    per_device_train_batch_size=2,           # 장치당 훈련 배치 크기
    per_device_eval_batch_size=2,            # 장치당 평가 배치 크기
    num_train_epochs=2,                       # 훈련할 에포크 수
    weight_decay=0.01,                        # 가중치 감쇠
)

# 미리 학습된 BERT 모델을 불러옵니다 (분류용).
model = NaverShoppingBert(768, 768, 4, model_name)

# Trainer 객체를 초기화합니다.
trainer = Trainer(
    model=model,                              # 사용할 모델
    args=training_args,                       # 훈련 매개변수
    train_dataset=train_dataset_shopping,              # 훈련 데이터셋
    eval_dataset=valid_dataset_shopping,                # 평가 데이터셋
)

# 모델을 훈련합니다.
trainer.train()

In [140]:
# NaverShoppingClassifier의 가중치를 저장
torch.save(model.naver_shopping_classifier.state_dict(), 'shopping_classifier.pth')


In [141]:
# 새롭게 model 생성
model = NaverShoppingBert(768, 768, 4, 'bert-base-multilingual-uncased')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [145]:
# 기존에 학습한 weight를 load
model.naver_shooping_classifier.load_state_dict(
    torch.load('shopping_classifier.pth')
)

C:\Users\USER\AppData\Local\Temp\ipykernel_23548\2738794209.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load('shopping_classifier.pth')


<All keys matched successfully>

In [384]:
# 저장한 weight를 load하여 추론
trainer.predict(test_dataset_shopping)

  0%|          | 0/250 [00:00<?, ?it/s]

PredictionOutput(predictions=array([[-0.04252329,  0.1618462 , -0.2530684 ,  0.18095714],
       [-0.03171977,  0.15301907, -0.25044414,  0.19558582],
       [-0.03076731,  0.14553234, -0.26409313,  0.20010293],
       ...,
       [-0.03610814,  0.15791234, -0.24591534,  0.18981424],
       [-0.04775867,  0.13160829, -0.25120008,  0.19475177],
       [-0.03156469,  0.14278744, -0.2520712 ,  0.19009817]],
      dtype=float32), label_ids=array([1, 3, 3, 0, 3, 1, 3, 3, 0, 0, 1, 0, 3, 2, 1, 0, 0, 3, 0, 3, 3, 0,
       2, 3, 3, 3, 1, 3, 1, 2, 3, 2, 0, 3, 3, 1, 1, 1, 3, 3, 3, 0, 1, 1,
       0, 3, 3, 3, 2, 3, 2, 3, 3, 3, 0, 3, 0, 2, 3, 3, 1, 3, 0, 1, 3, 0,
       3, 2, 3, 1, 3, 1, 0, 1, 3, 1, 3, 1, 1, 3, 3, 1, 1, 2, 0, 3, 2, 3,
       1, 3, 3, 0, 1, 3, 1, 3, 0, 3, 1, 2, 1, 1, 1, 3, 0, 0, 0, 0, 3, 0,
       3, 2, 3, 1, 1, 0, 2, 1, 0, 3, 3, 0, 1, 1, 3, 3, 3, 1, 3, 1, 3, 1,
       0, 3, 1, 1, 3, 1, 3, 3, 3, 3, 2, 1, 3, 1, 2, 3, 2, 1, 3, 3, 2, 3,
       3, 3, 3, 3, 0, 3, 0, 0, 0, 1, 0, 3, 1, 3, 

In [ ]:
# 새로운 adapter 생성
naver_movie_classifier = NaverMoviewClassifer(768, 768, 2, 'bert-base-multilingual-uncased')
naver_movie_classifier.load_state_dict(torch.load())

# 새로운 adapter 연결
model.naver_shooping_classifier = naver_movie_classifier

### LoRA

paper: https://arxiv.org/pdf/2106.09685


#### Introduction

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FYMCCt%2FbtsIYv4efkd%2FGVwpQOnQhc0c6AGNqwAI20%2Fimg.png" width="300" height="300"/>

<br>

<font style="font-size:20px"><b> 기존 fine-tuning의 문제점 </b></font>

1. 모델 크기 문제: fine-tuning 시 최종 모델도 원래 모델 만큼 많은 파라미터를 가짐 <br>
    -> 배포 문제: 특히 대규모 모델의 경우, 파라미터 수가 많아 배포가 어려워짐
2. Inference latency 증가: 모델 깊이 확장, 또는 시퀀스 길이를 줄이는 방식으로 인해 야기
    - 효율성과 품질 간의 trade-off


<br>

<font style="font-size:20px"><b> Contribution </b></font>

1. 모델 공유와 모듈화: 사전 학습된 모델을 공유하여 다양한 작업을 위한 작은 LoRA 모듈을 구축 가능. <br>
    -> 공유 모델을 고정하고, 작업 전환 시 그림과 같이 행렬 A와 B를 교체함으로써 저장 요구 사항과 작업 전환 오버헤드를 크게 줄일 수 있음.
2. 효율적인 훈련: LoRA는 훈련을 더 효율적으로 만들고 하드웨어 진입 장벽을 최대 3배 낮춤. <br>
    <- 대부분의 파라미터에 대해 기울기를 계산하거나 옵티마이저 상태를 유지할 필요가 없기 때문. <br>
    -> 훨씬 더 작은 저차 행렬만 최적화.
3. 지연 없는 배포: 배포 시 학습 가능한 행렬을 고정된 가중치와 병합할 수 있어, 완전히 파인튜닝된 모델과 비교해도 지연이 발생하지 않음.
4. 다양한 방법과의 호환성: LoRA는 많은 이전 방법들과 독립적이며, prefix-tuning과 같은 여러 방법과 결합 가능

#### Method

<font style="font-size:18px"><b> LOW-RANK-PARAMETRIZED UPDATE MATRICES </b></font>

특정 task에 fine-tuning할 때 사전 학습된 언어 모델은 더 작은 subspace로의 랜덤 projection에도 불구하고 여전히 효율적으로 학습할 수 있고, 낮은 instrisic dimension을 가짐 (https://arxiv.org/pdf/2012.13255) <br>
&nbsp;&nbsp;&nbsp;&nbsp; over-parametrized model은 실제로 low intrinsic dimension에 핵심이 존재 <br>
이에 영감받아 가중치에 대한 업데이트도 adaptation 중에 낮은 instrisic rank를 갖는다고 가정

$ W_0 + \Delta W = W_0 + B A $ <br>
&nbsp;&nbsp;&nbsp;&nbsp; where $ B \in \mathbb{R}^{d \times r}$, $A \in \mathbb{R}^{r \times k}$, $r \ll \min(d, k) $, r: rank, $W_0$: pre-trained weight <br>

$ h = w_0x + \Delta x = W_0x + BAx $
- $ \Delta Wx$: $ \alpha / r $로 스케일링
- $ \alpha $: r의 상수

<br>

<font style="font-size:18px"><b> APPLYING LORA TO TRANSFORMER </b></font>

![](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2Focf8S%2FbtsIYWNUe7c%2Fv60Vk4pQj3AnpwrjGTPb6k%2Fimg.png)

weight 행렬의 모든 부분집합에 LoRA 적용 가능. <br>
&nbsp;&nbsp;&nbsp;&nbsp;단순성 및 파라미터 효율성을 위해 task에 대한 attention 내 weight만 튜닝하고 mlp 모듈은 고정. <br>
-> r << $d_{\text{model}}$의 경우 VARM을 최대 2/3까지 줄일 수 있음 <br>
&nbsp;&nbsp;&nbsp;&nbsp;    ex) 1.2T -> 350G (GPT3) <br>
-> $r=4$이고, q, k만 튜닝되면 모델 사이즈가 350GB -> 35MB로 감소 <br>
모든 parameter가 아닌 LoRA 가중치면 교환하기에 더 저렴한 비용으로 배포 중 task 전환이 용이. <br>
대다수의 파라미터에서 기울기 재계산이 필요없기에 전체 fine-tuning대비 속도 향상. <br>
&nbsp;&nbsp;&nbsp;&nbsp; ex) GPT3 - 25%


#### 사용 방법

> ```python
> model_name = 'bert-base-uncased'
> model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)
> tokenizer = AutoTokenizer.from_pretrained(model_name)
> 
> lora_config = LoraConfig(
>     r=8,  
>     lora_alpha=16,
>     lora_dropout=0.1,
>     task_type=TaskType.SEQ_CLS,
>     target_modules=[<layer_name>],
> )
> 
> lora_model = get_peft_model(model, lora_config)
> lora_model.print_trainable_parameters()
> ```
- r: Low-rank
- lora_alpha: Scaling factor
- lora_dropout: Dropout rate
- task_type: task type


In [198]:
# 사전 훈련된 BERT 모델을 로드
base_model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased')

# LoRA 구성 설정
lora_config = LoraConfig(
    r=4,  # Low-rank 매트릭스의 차원
    lora_alpha=16,  # LoRA에서 사용되는 스케일링 계수
    lora_dropout=0.1,  # Dropout 비율
    task_type=TaskType.SEQ_CLS,  # 작업 유형: 시퀀스 분류
    target_modules=['query', 'key'],  # LoRA 적용할 모듈
)

# LoRA 모델 생성
model = get_peft_model(base_model, lora_config)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.


In [197]:
base_model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [199]:
model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): BertForSequenceClassification(
      (bert): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0-11): 12 x BertLayer(
              (attention): BertAttention(
                (self): BertSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default

In [244]:
# 사전 훈련된 OpenAI GPT 모델 로드
base_model = AutoModelForCausalLM.from_pretrained('openai-community/openai-gpt')

# LoRA 구성 설정
lora_config = LoraConfig(
    r=4,  # Low-rank 매트릭스의 차원
    lora_alpha=16,  # LoRA에서 사용되는 스케일링 계수
    lora_dropout=0.1,  # Dropout 비율
    task_type=TaskType.CAUSAL_LM,  # 작업 유형: 인과 언어 모델링
    target_modules=['c_attn', 'c_proj'],  # LoRA 적용할 모듈
)

# LoRA 모델 생성
model = get_peft_model(base_model, lora_config)

C:\Users\USER\AppData\Roaming\Python\Python310\site-packages\peft\tuners\lora\layer.py:1150: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [206]:
model.print_trainable_parameters()

trainable params: 405,504 || all params: 116,940,288 || trainable%: 0.3468


In [245]:
summary(model)

Layer (type:depth-idx)                                            Param #
PeftModelForCausalLM                                              --
├─LoraModel: 1-1                                                  --
│    └─OpenAIGPTLMHeadModel: 2-1                                  --
│    │    └─OpenAIGPTModel: 3-1                                   116,940,288
│    │    └─Linear: 3-2                                           (31,087,104)
Total params: 148,027,392
Trainable params: 405,504
Non-trainable params: 147,621,888

In [ ]:
## day46 bert NLI 
## bert 모델에 LoRA (q, k) 적용해서 결과 보기
## Colab L4

def preprocessing(row):
    # 주어진 행(row)에서 전제(premise)와 가설(hypothesis)을 토크나이즈합니다.
    return tokenizer(
        row['premise'],                          # 전제 문장
        row['hypothesis'],                       # 가설 문장
        truncation=True,                        # 길이가 max_length를 초과하는 경우 잘라냄
        padding='max_length',                           # 최대 길이에 맞춰 패딩 추가
        return_tensors='pt',                   # 파이토치 텐서로 반환
        max_length=128,                        # 최대 길이 설정
    )

dataset = load_dataset('snli')

# 훈련 데이터셋에서 처음 1000개의 샘플을 선택합니다.
train_dataset = dataset['train'].select(range(2000))

# 검증 데이터셋에서 처음 1000개의 샘플을 선택합니다.
valid_dataset = dataset['validation'].select(range(2000))

# BERT 모델의 이름을 지정합니다. 여기서는 소문자 처리가 된 BERT 모델을 사용합니다.
model_name = 'bert-base-uncased'

# 미리 학습된 BERT 토크나이저를 로드합니다.
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 훈련 데이터셋에 대해 전처리 함수를 적용하여 토크나이즈된 데이터셋을 생성합니다.
tokenized_train_dataset = train_dataset.map(preprocessing, batched=True)

# 검증 데이터셋에 대해 전처리 함수를 적용하여 토크나이즈된 데이터셋을 생성합니다.
tokenized_valid_dataset = valid_dataset.map(preprocessing, batched=True)

# 토크나이즈된 훈련 데이터셋을 리스트로 변환하며, 레이블이 -1이 아닌 데이터만 포함합니다.
tokenized_train_dataset = [
    {'input_ids': row.get('input_ids'),                # 입력 ID
     'token_type_ids': row.get('token_type_ids'),      # 토큰 타입 ID
     'attention_mask': row.get('attention_mask'),       # 어텐션 마스크
     'label': row.get('label')}                          # 레이블
    for row in tokenized_train_dataset
    if row.get('label') != -1                          # 레이블이 -1이 아닌 경우만 포함
]

# 토크나이즈된 검증 데이터셋을 리스트로 변환하며, 레이블이 -1이 아닌 데이터만 포함합니다.
tokenized_valid_dataset = [
    {'input_ids': row.get('input_ids'),                # 입력 ID
     'token_type_ids': row.get('token_type_ids'),      # 토큰 타입 ID
     'attention_mask': row.get('attention_mask'),       # 어텐션 마스크
     'label': row.get('label')}                          # 레이블
    for row in tokenized_valid_dataset
    if row.get('label') != -1                          # 레이블이 -1이 아닌 경우만 포함
]

class NLIDataset(Dataset):
    def __init__(self, data):
        # 데이터셋 초기화: 주어진 데이터를 저장합니다.
        self.data = data
    
    def __len__(self):
        # 데이터셋의 크기를 반환합니다.
        return len(self.data)

    def __getitem__(self, idx):
        # 주어진 인덱스(idx)에 해당하는 데이터 항목을 반환합니다.
        return {
            'input_ids': self.data[idx].get('input_ids'),          # 입력 ID
            'token_type_ids': self.data[idx].get('token_type_ids'),# 토큰 타입 ID
            'attention_mask': self.data[idx].get('attention_mask'), # 어텐션 마스크
            'labels': self.data[idx].get('label'),                  # 레이블
        }

# 토크나이즈된 훈련 데이터셋을 NLIDataset 클래스를 사용하여 데이터셋 객체로 변환합니다.
train_dataset = NLIDataset(tokenized_train_dataset)

# 토크나이즈된 검증 데이터셋을 NLIDataset 클래스를 사용하여 데이터셋 객체로 변환합니다.
valid_dataset = NLIDataset(tokenized_valid_dataset)

# 학습을 위한 파라미터 설정
training_args = TrainingArguments(
    output_dir='./results/NLI',                   # 결과 저장 디렉토리
    eval_strategy='epoch',                         # 평가 전략: 매 에폭마다 평가
    learning_rate=2e-5,                           # 학습률
    warmup_steps=50,                              # 웜업 단계 수
    per_device_train_batch_size=256,               # 디바이스당 훈련 배치 크기
    per_device_eval_batch_size=256,                # 디바이스당 평가 배치 크기
    num_train_epochs=1,                           # 훈련 에폭 수
    weight_decay=0.01,                            # 가중치 감소
    logging_dir='./logs',                         # 로그 저장 디렉토리
)

# 미리 학습된 BERT 모델을 로드하고, 레이블 수를 설정합니다. 여기서는 3개의 레이블이 있습니다.
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

lora_config = LoraConfig(
    r=4,                       # LoRA의 차원 수
    lora_alpha=2e-5,          # LoRA의 학습률 조정값
    lora_dropout=0.1,          # 드롭아웃 비율
    task_type=TaskType.SEQ_CLS, # 작업 유형: 시퀀스 분류
    target_modules=['query', 'key'] # LoRA를 적용할 모듈
)
model = get_peft_model(base_model, lora_config) # LoRA 설정이 적용된 모델을 생성

# Trainer 객체를 생성하여 모델, 학습 인자, 훈련 데이터셋, 평가 데이터셋을 지정합니다.
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,                  # 훈련 데이터셋
    eval_dataset=valid_dataset,                   # 검증 데이터셋
)

# 모델을 훈련합니다.
trainer.train()

# 훈련된 모델을 저장합니다.
model.save_pretrained('./bert_nli')

## Quantization


<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbXst46%2FbtrLgq8tGwo%2FwMWbieyztBgOz4RvkEW9C1%2Fimg.png" width="400">

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FO8PQn%2FbtrLgN9Y8gp%2FhnXOuZ19p7KOnG023WbATk%2Fimg.png" width="400">


paper: https://arxiv.org/pdf/1712.05877

<br>

모델 parameter를 low bit로 표현하여 모델 사이즈를 줄이는 경량화 기법 <br>
&nbsp;&nbsp;&nbsp;&nbsp; fp32 -> int8

<br>

<font style="font-size:18px"> 분류 </font>

- fixed point dnn quantization: inference에 사용할 model을 만드는 것을 목적으로 함 <br>
-> forward에 대해서만 quantization


<center><img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbNUGim%2FbtsG2VXn5AC%2F2JcO58bbR7xz5KYsB8qODK%2Fimg.png" width="400"></center>


- floating point dnn quantization: backward에 대해서도 quantization 

<br>

<font style="font-size:18px"> 사용 방법 </font>

> ```python
> quantization_config = BitsAndBytesConfig(
>     load_in_8bit=True,  # 8 bit 양자화
>     load_in_4bit=True,  # 4 bit 양자화
> )
> 
> model = AutoModelForCausalLM.from_pretrained(
>     <model_name>, 
>     quantization_config=quantization_config,
> )
> 
> model.get_memory_footprint()  # 용량 확인 (byte 단위)
>
> model.dequantize()  # 비양자화
>
> ```

In [ ]:
# 1. 양자화 설정
# 모델을 4비트로 로드할 수 있도록 BitsAndBytesConfig 설정
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 4비트로 모델을 로드
)

# 2. 기본 모델 로드
# 양자화 설정을 적용하여 LLaMA 모델을 로드
base_model = AutoModelForCausalLM.from_pretrained(
    'meta-llama/Llama-3.2-3B-Instruct',  # 사전 훈련된 LLaMA 모델
    quantization_config=quantization_config,  # 위에서 설정한 양자화 설정 적용
)

# 3. LoRA 설정
# LoRA를 적용하기 위한 설정
lora_config = LoraConfig(
    r=2,                   # LoRA의 차원 수 (저차원 표현)
    lora_alpha=1e-4,      # LoRA의 학습률 조정값 (작은 값으로 설정)
    lora_dropout=0.1,      # 드롭아웃 비율 (과적합 방지를 위해 설정)
    task_type=TaskType.CAUSAL_LM, # 작업 유형: 언어 생성
    target_modules=['q_proj', 'p_proj'],  # LoRA를 적용할 모듈 지정 (쿼리와 프로젝션 모듈)
)

# 4. LoRA 모델 생성
# 기본 모델과 LoRA 설정을 통합하여 LoRA 모델 생성
model = get_peft_model(base_model, lora_config)